# CREACION BASE DE DATOS

In [ ]:
# IMPORTAR LIBRERIAS
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
 #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np #para convertir formato nan de Paython a none de MySQL
import mysql.connector #para conectar Paython con MySQL
from mysql.connector import Error # para capturar errores de MySQL


load_dotenv() # Carga las variables definidas en el archivo .env // Devuelve True si se cargan correctamente, False si no
host=os.getenv("MYSQL_HOST") # Obtiene el valor de la variable de entorno MYSQL_HOST
user=os.getenv("MYSQL_USER") # Obtiene el usuario de la base de datos
password=os.getenv("MYSQL_PASSWORD") # Obtiene la contraseña de la base de datos

In [ ]:
# FASE 2: CREACIÓN DE LA BASE DE DATOS (VERSIÓN PYTHON)

# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='contraseña',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema

In [ ]:
# B. Creación de la base de datos

try:
    # Creamos un cursor: es el intermediario entre Python y MySQL, necesario para ejecutar consultas SQL
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    # "IF NOT EXISTS" evita errores si la base de datos ya está creada
    query_create_database = "CREATE DATABASE IF NOT EXISTS MusicStream_db"

    # Ejecutamos la consulta SQL para crear la base de datos
    mycursor.execute(query_create_database)
    print("Base de datos creada correctamente (o ya existía).")

# Captura errores relacionados con MySQL (por ejemplo, permisos insuficientes)
except mysql.connector.Error as e:  
    print("Ocurrió un error al crear la base de datos:", e)

In [ ]:
# C. Inserción de datos
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s)
"""

try:
    columnas_a_insertar = ["nombre","album","genero","año"]# indica que columnas quiero insertar del DataFrame
    df_filtrado = canciones_2016_df[columnas_a_insertar]

    df_corregido_SPOTIFY = df_filtrado.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

In [ ]:
# CREACION DE TABLA INTERMEDIA QUE RELACIONA CANCIONES Y ARTISTAS 

mycursor.execute("USE MusicStream_db")

query = '''CREATE TABLE IF NOT EXISTS Cancion_artista(
	ID_Cancion INT,
    ID_Artista INT,
    PRIMARY KEY (ID_Cancion,ID_Artista),
    FOREIGN KEY (ID_Cancion) REFERENCES Canciones (ID_Cancion),
    FOREIGN KEY (ID_Artista) REFERENCES Artistas (Id_Artista)
);'''
mycursor.execute(query)

In [ ]:
# INSERTAR DATOS EN TABLA Cancion_Artista

cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database="MusicStream_db"
)
mycursor = cnx.cursor(buffered=True)

try:
    # Crear lista vacía para guardar las relaciones
    relaciones = []

    # Recorremos el DataFrame fila por fila
    for _, fila in canciones_2014_df.iterrows():
        
        # Obtener ID_Cancion desde MySQL
        mycursor.execute(
            "SELECT ID_Cancion FROM Canciones WHERE Nombre = %s",
            (fila["nombre"],)
        )
        cancion = mycursor.fetchone()
        if cancion:
            id_cancion = cancion[0]
        else:
            continue  # Si no encuentra la canción, saltar esta fila

        # Obtener ID_Artista desde MySQL
        mycursor.execute(
            "SELECT ID_Artista FROM Artistas WHERE Artista = %s",
            (fila["artista"],)
        )
        artista = mycursor.fetchone()
        if artista:
            id_artista = artista[0]
        else:
            continue  # Si no encuentra el artista, saltar esta fila

        # Añadir la tupla (id_cancion, id_artista) a la lista
        relaciones.append((id_cancion, id_artista))

    # Insertar todas las relaciones de golpe usando INSERT IGNORE
    query_insert = """
    INSERT IGNORE INTO Cancion_artista (ID_Cancion, ID_Artista)
    VALUES (%s, %s)
    """
    mycursor.executemany(query_insert, relaciones)
    cnx.commit()

    print(f"{mycursor.rowcount} relaciones insertadas en Cancion_artista (IGNORE aplicado)")

except mysql.connector.Error as e:
    print("Error al insertar relaciones en Cancion_artista:", e)
    cnx.rollback()  # revertir cambios si hay error



In [ ]:
cnx.close()
